In [1]:
from sctoolbox.utilities import bgcolor

# Marker genes and differentially expressed genes between groups
<hr style="border:2px solid black"> </hr>

## Get marker genes per cluster

The marker genes of a cluster are defined as differentially expressed genes of that cluster compared to all other cells/clusters. These genes are typically used to annotate the cluster to a specific cell type.

The gene ranking uses the rank_genes_groups() method from scanpy. See detailed documentation [here](https://scanpy.readthedocs.io/en/stable/generated/scanpy.tl.rank_genes_groups.html).

---------

## Loading packages

In [ ]:
import scanpy as sc
import pandas as pd
import sctoolbox.utilities as utils
import sctoolbox.tools as tools
import sctoolbox.plotting as pl
import matplotlib.backends.backend_pdf
utils.settings_from_config("config.yaml", key="05")

## Loading adata

In [ ]:
adata = utils.load_h5ad("anndata_4.h5ad")

with pd.option_context("display.max.rows", 5, "display.max.columns", None):
    display(adata)
    display(adata.obs)
    display(adata.var)

<hr style="border:2px solid black"> </hr>

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [2]:
%bgcolor PowderBlue

# Column names of clustering columns
clustering_cols = ["leiden_0.1", "leiden_0.2", "leiden_0.3", "leiden_0.4", "leiden_0.5",
                   "leiden_0.6", "leiden_0.7", "leiden_0.8", "leiden_0.9"]

# Marker genes

# Method for gene ranking
ranking_method = "t-test"
# Top X genes to be reported, None for all genes
n_genes = None

# Marker gene filter

# Minimum fraction of cells in a group that must express a gene to be considered as a marker gene
min_in_group_fraction = 0.25
# Minimum foldchange (+/-) to be considered as a marker gene
min_fold_change = 0.5
# Maximum fraction of cells in other groups that must express a gene to be considered as a marker gene
max_out_group_fraction = 0.8

# Plotting
n_genes_markerplot = 15
marker_style = "dots"  # Either `dots` or `heatmap`.

<hr style="border:2px solid black"> </hr>

## Automatic markers per cluster using rank_genes_groups

In [ ]:
marker_tables = dict()
for clustering in clustering_cols:
    
    # Identify markers per cluster (adjust group fraction and fold change to filter genes)
    tools.run_rank_genes(adata, clustering,
                         min_in_group_fraction=min_in_group_fraction,
                         min_fold_change=min_fold_change,
                         max_out_group_fraction=max_out_group_fraction,
                         n_genes=n_genes,
                         ranking_method=ranking_method)
    
    # Plot dotplot of markers
    _ = pl.rank_genes_plot(adata, key=f"rank_genes_{clustering}_filtered",
                           n_genes=n_genes_markerplot, style=marker_style,
                           save=f"marker_genes_{style}_{clustering}.pdf")

    # Write marker genes to table
    marker_table = tools.get_rank_genes_tables(adata, out_group_fractions=True,
                                               key=f"rank_genes_{clustering}_filtered",
                                               save_excel=f"cluster_marker_genes_{clustering}.xlsx")
    marker_tables[clustering] = marker_table

---------

# Plot expression

This section of the notebook plots the marker gene expression for each selected clustering in the selected embedding scatterplot. There is also an option to specify a custom list of genes to be plotted.

<hr style="border:2px solid black"> </hr>

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [3]:
%bgcolor PowderBlue

# Column names of clustering columns
marker_table_cols = ["leiden_0.1", "leiden_0.2", "leiden_0.3", "leiden_0.4", "leiden_0.5",
                     "leiden_0.6", "leiden_0.7", "leiden_0.8", "leiden_0.9"]
n_marker = 5
n_cols = 5

# List of genes which should be plotted instead
custom_gene_list = []

embedding = "umap"  # Either umap or tsne

<hr style="border:2px solid black"> </hr>

In [ ]:
if embedding == "umap":
    pl_method = sc.pl.umap
elif embedding == "tsne":
    pl_method = sc.pl.tsne
else:
    raise ValueError("Invalid embedding set.")

## Plot custom gene list

In [ ]:
if custom_gene_list:
    pl_method(adata, color=custom_gene_list, cmap=pl.sc_colormap(), ncols=n_cols)
    pl._save_figure(f"{embedding}_custom_gene_list_expression.pdf")

### Plot cluster marker

In [ ]:
for cluster_col in marker_table_cols:
    pdf = matplotlib.backends.backend_pdf.PdfPages(f"{utils.settings.figure_dir}/{embedding}_clustering_{cluster_col}_expression.pdf")
    for cluster, table in marker_tables[cluster_col].items():
        marker = list(table["names"][:n_marker])
        title = [f"Cluster_{cluster} - {gene}" for gene in marker]
        fig = sc.pl.embedding(adata, basis="X_" + embedding, color=marker, show=False,
                              ncols=n_cols, title=title, return_fig=True)
        pdf.savefig(fig)
    pdf.close()

-------------

# DEG between conditions

 ## Run DEseq2 between conditions/clusters

In [ ]:
# Normalize raw counts across cells
d = sc.pp.normalize_total(adata, layer="raw", inplace=False) # returns a dict
adata.layers["raw_norm"] = d["X"]
adata.layers["raw_norm"] = adata.layers["raw_norm"].ceil().astype(int)

In [4]:
%bgcolor PowderBlue

# Adjust which columns to use for DEseq2
sample_col = "sample"
condition_col = "chamber"

In [ ]:
# Run DEseq2
deseq_table = tools.run_deseq2(adata, sample_col, condition_col, layer="raw_norm")

In [ ]:
deseq_table.head(10)

--------------

## Save adata

In [ ]:
#fix error when saving filtered rank gene names
import re
for key in list(adata.uns.keys()):
    if re.match("rank_genes_.*_filtered", key):
        del adata.uns[key]

In [ ]:
adata

In [ ]:
utils.save_h5ad(adata, "anndata_5.h5ad")